In [1]:
corpus_list = ['BibleTTS/akuapem-twi',
    'BibleTTS/asante-twi',
    'BibleTTS/ewe',
    'BibleTTS/hausa',
    'BibleTTS/lingala'
    'BibleTTS/yoruba',
    'Buckeye',
    'EUROM',
    'LibriSpeech',
    'LibriVox',
    'MediaSpeech/AR',
    'MediaSpeech/ES',
    'MediaSpeech/FR',
    'MediaSpeech/TR',
    'MozillaCommonVoice/ab',
    'MozillaCommonVoice/ar',
    'MozillaCommonVoice/ba',
    'MozillaCommonVoice/be',
    'MozillaCommonVoice/bg',
    'MozillaCommonVoice/bn',
    'MozillaCommonVoice/br',
    'MozillaCommonVoice/ca',
    'MozillaCommonVoice/ckb',
    'MozillaCommonVoice/cnh',
    'MozillaCommonVoice/cs',
    'MozillaCommonVoice/cv',
    'MozillaCommonVoice/cy',
    'MozillaCommonVoice/da',
    'MozillaCommonVoice/de',
    'MozillaCommonVoice/dv',
    'MozillaCommonVoice/el',
    'MozillaCommonVoice/en',
    'MozillaCommonVoice/eo',
    'MozillaCommonVoice/es',
    'MozillaCommonVoice/et',
    'MozillaCommonVoice/eu',
    'MozillaCommonVoice/fa',
    'MozillaCommonVoice/fi',
    'MozillaCommonVoice/fr',
    'MozillaCommonVoice/fy-NL',
    'MozillaCommonVoice/ga-IE',
    'MozillaCommonVoice/gl',
    'MozillaCommonVoice/gn',
    'MozillaCommonVoice/hi',
    'MozillaCommonVoice/hu',
    'MozillaCommonVoice/hy-AM',
    'MozillaCommonVoice/id',
    'MozillaCommonVoice/ig',
    'MozillaCommonVoice/it',
    'MozillaCommonVoice/ja',
    'MozillaCommonVoice/ka',
    'MozillaCommonVoice/kab',
    'MozillaCommonVoice/kk',
    'MozillaCommonVoice/kmr',
    'MozillaCommonVoice/ky',
    'MozillaCommonVoice/lg',
    'MozillaCommonVoice/lt',
    'MozillaCommonVoice/ltg',
    'MozillaCommonVoice/lv',
    'MozillaCommonVoice/mhr',
    'MozillaCommonVoice/ml',
    'MozillaCommonVoice/mn',
    'MozillaCommonVoice/mt',
    'MozillaCommonVoice/nan-tw',
    'MozillaCommonVoice/nl',
    'MozillaCommonVoice/oc',
    'MozillaCommonVoice/or',
    'MozillaCommonVoice/pl',
    'MozillaCommonVoice/pt',
    'MozillaCommonVoice/ro',
    'MozillaCommonVoice/ru',
    'MozillaCommonVoice/rw',
    'MozillaCommonVoice/sr',
    'MozillaCommonVoice/sv-SE',
    'MozillaCommonVoice/sw',
    'MozillaCommonVoice/ta',
    'MozillaCommonVoice/th',
    'MozillaCommonVoice/tr',
    'MozillaCommonVoice/tt',
    'MozillaCommonVoice/ug',
    'MozillaCommonVoice/uk',
    'MozillaCommonVoice/ur',
    'MozillaCommonVoice/uz',
    'MozillaCommonVoice/vi',
    'MozillaCommonVoice/yo',
    'MozillaCommonVoice/yue',
    'MozillaCommonVoice/zh-CN',
    'MozillaCommonVoice/zh-TW',
    'primewords_chinese',
    'room_reader',
    'SpeechClarity',
    'TAT-Vol2',
    'thchs30',
    'TIMIT',
    'TTS_Javanese',
    'zeroth_korean',]

In [7]:
import scipy.io as sio
import pandas as pd
import numpy as np
import glob
import csv
import json


def make_meta_file(corpus):
    
    params_list = glob.glob('STM_output/Survey/speech_params_'+corpus+'/*')
    df_list = []

    # load the data from the mat file
    for params_file in params_list:
        data_dict = sio.loadmat(params_file)
        structure_dict = {field: data_dict['Params'][field][0] for field in data_dict['Params'].dtype.names}
        df = pd.DataFrame(structure_dict)
        df.drop(columns=['x_axis','y_axis'], inplace=True)
        df = df.applymap(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
        df = df.applymap(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
        df_list.append(df)
    
    df_all = pd.concat(df_list, ignore_index=True)

    # add the speaker ID and gender info
    if 'MozillaCommonVoice' in corpus:
        valid_df = pd.read_csv('data/speechCorp/'+corpus+'/validated.tsv', sep='\t', quoting=csv.QUOTE_NONE)
        valid_df['path'] = valid_df['path'].str.replace('.mp3', '')
        valid_df.rename(columns={'client_id':'speaker/player'}, inplace=True)
        df_all = df_all.merge(valid_df[['speaker/player', 'path', 'gender']], how='left', left_on='filename', right_on='path').drop(columns=['path'])
    elif 'BibleTTS' in corpus:
        df_all['speaker/player'] = 'BibleTTS_' + df_all['filename'].str[:3]
        df_all['gender'] = np.nan
    elif 'Buckeye' in corpus:
        df_all['speaker/player'] = df_all['filename'].str[:3].str.replace('s', 'S')
        df_gender = pd.read_csv('data/speechCorp/Buckeye/Buckeye_speaker_info.csv')
        df_all = df_all.merge(df_gender[['SPEAKER', "SPEAKER'S GENDER"]], how='left', left_on='speaker/player', right_on='SPEAKER').drop(columns=['SPEAKER'])
        df_all.rename(columns={"SPEAKER'S GENDER":'gender'}, inplace=True)
        df_all['speaker/player'] = 'Buckeye_'+df_all['speaker/player']
    elif 'EUROM' in corpus:
        df_all['speaker/player'] = 'EUROM_' + df_all['LangOrInstru'] +'_'+ df_all['filename'].str[:2]
        df_all['gender'] = np.nan
    elif 'MediaSpeech' in corpus:
        df_all['speaker/player'] = 'MediaSpeech_' + df_all['LangOrInstru'] +'_'+ df_all['filename']
        df_all['gender'] = np.nan
    elif 'LibriSpeech' in corpus:
        split_names = df_all['filename'].str.split('-') # Split the 'name' column by "-"
        first_parts = split_names.str[0] # Extract the first part of the split result
        df_all['speaker/player'] = first_parts # Add the extracted part as a new column in the DataFrame
        
        ## load LibriSpeech text file
        reader_ids = []
        genders = []
        subsets = []
        durations = []
        names = []
        # Open the file and read line by line
        with open('data/speechCorp/LibriSpeech/SPEAKERS.TXT', 'r') as file:
            # Skip lines starting with ";" (comments) until reaching the data
            while True:
                line = file.readline()
                if not line.startswith(';'):
                    # Start processing data from this line
                    break
            
            # Read the rest of the lines and parse them
            while line:
                # Split each line by "|" character
                data = line.strip().split('|')
                # Extract relevant information
                reader_ids.append(int(data[0].strip()))
                genders.append(data[1].strip())
                subsets.append(data[2].strip())
                durations.append(float(data[3].strip()))
                names.append(data[4].strip())
                # Read the next line
                line = file.readline()
        
        # Create a DataFrame using the lists
        df_LibriSpeech = pd.DataFrame({'reader_id': reader_ids, 'gender': genders, 'subset': subsets, 'duration': durations, 'name': names})
        df_LibriSpeech['reader_id']= df_LibriSpeech['reader_id'].astype(str)
        df_all = df_all.merge(df_LibriSpeech[['reader_id', 'gender']], how='left', left_on='speaker/player', right_on='reader_id').drop(columns=['reader_id'])
    elif 'primewords_chinese' in corpus:
        with open('data/speechCorp/primewords_chinese/set1_transcript.json', 'r') as file:
            data = json.load(file)
        primewords_df = pd.DataFrame(data)
        primewords_df['file'] = primewords_df['file'].str.replace('.wav', '')
        df_all = df_all.merge(primewords_df[['file', 'user_id']], how='left', left_on='filename', right_on='file').drop(columns=['file'])
        df_all.rename(columns={"user_id":'speaker/player'}, inplace=True)
        df_all['speaker/player'] = 'primewords_'+df_all['speaker/player']
        df_all['gender']=np.nan
    elif 'room_reader' in corpus:
        split_names = df_all['filename'].str.split('_') # Split the 'name' column by "-"
        df_all['speaker/player'] = split_names.str[1] # Extract the first part of the split result
        RR_df = pd.read_excel('data/speechCorp/room_reader/RoomReader_SessionsEvents.xlsx')
        df_all = df_all.merge(RR_df[['part_ID', 'gender']], how='left', left_on='speaker/player', right_on='part_ID').drop(columns=['part_ID'])
        df_all['speaker/player'] = 'RoomReader_'+df_all['speaker/player']
    elif 'SpeechClarity' in corpus:
        df_all['speaker/player'] = 'SpeechClarity_'+df_all['filename'].str[:3]
        df_all['gender'] = np.nan
    elif 'TAT-Vol2' in corpus:
        df_all['speaker/player'] = 'TAT-Vol2_'+df_all['filename'].str[:10]
        df_all['gender'] = df_all['filename'].str[5]


    return df_all

In [16]:
corpus = 'MozillaCommonVoice/hi'
df_all = make_meta_file(corpus)
df_all

,filepath,filename,LangOrInstru,VoiOrNot,startPoint,endPoint,FSkhz,expdur,maxsil,totalLengCur,speaker/player,gender
0,data/speechCorp/MozillaCommonVoice/hi/clips/co...,common_voice_hi_26986130,Hindi,1,1,128000,32,4,0.903469,4,d24af5ee15190d3fc14dfc4ee3c75c75fe66fb537d4c7c...,male
1,data/speechCorp/MozillaCommonVoice/hi/clips/co...,common_voice_hi_26021896,Hindi,1,1,128000,32,4,0.666188,4,0f018a99663f33afbb7d38aee281fb1afcfd07f9e7acd0...,NaN
2,data/speechCorp/MozillaCommonVoice/hi/clips/co...,common_voice_hi_24026258,Hindi,1,1,128000,32,4,0.668438,4,63293ff4cc67280b10da2df2fd2763db65ca8493af191e...,male
3,data/speechCorp/MozillaCommonVoice/hi/clips/co...,common_voice_hi_25170240,Hindi,1,1,128000,32,4,0.620031,4,c88824395561074e0eb0ebc1280d4866dbc29f01c02689...,NaN
4,data/speechCorp/MozillaCommonVoice/hi/clips/co...,common_voice_hi_23849297,Hindi,1,1,192000,48,4,0.792937,4,46f23ae11e8565e979749342e1097a77fe3ef574253f10...,male
...,...,...,...,...,...,...,...,...,...,...,...,...
5433,data/speechCorp/MozillaCommonVoice/hi/clips/co...,common_voice_hi_27761274,Hindi,1,1,128000,32,4,0.164656,4,d5880664e0b201cf24b4972ea9890b801afe05af05cd43...,male
5434,data/speechCorp/MozillaCommonVoice/hi/clips/co...,common_voice_hi_25241159,Hindi,1,1,128000,32,4,0.803937,4,3255dc44d5426ad255e14c4db4d12b224dfffd1e594a40...,male
5435,data/speechCorp/MozillaCommonVoice/hi/clips/co...,common_voice_hi_27517919,Hindi,1,1,128000,32,4,0.212375,4,d5880664e0b201cf24b4972ea9890b801afe05af05cd43...,male
5436,data/speechCorp/MozillaCommonVoice/hi/clips/co...,common_voice_hi_32264979,Hindi,1,1,256000,32,8,0.392625,8,b0900acc53b0b578a071411f8ec0479ebf48be094bdcae...,NaN


## test code ##

In [88]:
corpus = 'TAT-Vol2'
params_list = glob.glob('STM_output/Survey/speech_params_'+corpus+'/*')
df_list = []

for params_file in params_list:
    data_dict = sio.loadmat(params_file)
    structure_dict = {field: data_dict['Params'][field][0] for field in data_dict['Params'].dtype.names}
    df = pd.DataFrame(structure_dict)
    df.drop(columns=['x_axis','y_axis'], inplace=True)
    df = df.applymap(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
    df = df.applymap(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
    df_list.append(df)

df_all = pd.concat(df_list, ignore_index=True)
df_all.head()

,filepath,filename,LangOrInstru,VoiOrNot,startPoint,endPoint,FSkhz,expdur,maxsil,totalLengCur
0,data/speechCorp/TAT-Vol2/condenser/TH_THM0018/...,TH_THM0018_0029-5.59-03,Taiwanese,1,1,128000,16,8,0.001063,4
1,data/speechCorp/TAT-Vol2/condenser/TS_TSF0017/...,TS_TSF0017_0035-4.24-03,Taiwanese,1,1,64000,16,4,0.106875,4
2,data/speechCorp/TAT-Vol2/condenser/TH_THM0018/...,TH_THM0018_0029-3.51-03,Taiwanese,1,1,128000,16,8,0.487187,8
3,data/speechCorp/TAT-Vol2/condenser/TS_TSF0017/...,TS_TSF0017_0035-3.6-03,Taiwanese,1,1,64000,16,4,0.102813,4
4,data/speechCorp/TAT-Vol2/condenser/TH_THM0018/...,TH_THM0018_0029-6.2-03,Taiwanese,1,1,64000,16,4,0.021687,4


In [89]:
df_all['speaker/player'] = 'TAT-Vol2_'+df_all['filename'].str[:10]
df_all['gender'] = df_all['filename'].str[5]
df_all

,filepath,filename,LangOrInstru,VoiOrNot,startPoint,endPoint,FSkhz,expdur,maxsil,totalLengCur,speaker/player,gender
0,data/speechCorp/TAT-Vol2/condenser/TH_THM0018/...,TH_THM0018_0029-5.59-03,Taiwanese,1,1,128000,16,8,0.001063,4,TAT-Vol2_TH_THM0018,M
1,data/speechCorp/TAT-Vol2/condenser/TS_TSF0017/...,TS_TSF0017_0035-4.24-03,Taiwanese,1,1,64000,16,4,0.106875,4,TAT-Vol2_TS_TSF0017,F
2,data/speechCorp/TAT-Vol2/condenser/TH_THM0018/...,TH_THM0018_0029-3.51-03,Taiwanese,1,1,128000,16,8,0.487187,8,TAT-Vol2_TH_THM0018,M
3,data/speechCorp/TAT-Vol2/condenser/TS_TSF0017/...,TS_TSF0017_0035-3.6-03,Taiwanese,1,1,64000,16,4,0.102813,4,TAT-Vol2_TS_TSF0017,F
4,data/speechCorp/TAT-Vol2/condenser/TH_THM0018/...,TH_THM0018_0029-6.2-03,Taiwanese,1,1,64000,16,4,0.021687,4,TAT-Vol2_TH_THM0018,M
...,...,...,...,...,...,...,...,...,...,...,...,...
788,data/speechCorp/TAT-Vol2/condenser/TH_THM0018/...,TH_THM0018_0029-3.37-03,Taiwanese,1,1,128000,16,8,0.250312,8,TAT-Vol2_TH_THM0018,M
789,data/speechCorp/TAT-Vol2/condenser/TS_TSF0017/...,TS_TSF0017_0035-4.42-03,Taiwanese,1,1,64000,16,4,0.113250,4,TAT-Vol2_TS_TSF0017,F
790,data/speechCorp/TAT-Vol2/condenser/TS_TSM0020/...,TS_TSM0020_0035-3.71-03,Taiwanese,1,1,128000,16,8,0.017438,8,TAT-Vol2_TS_TSM0020,M
791,data/speechCorp/TAT-Vol2/condenser/TS_TSF0017/...,TS_TSF0017_0035-5.65-03,Taiwanese,1,1,128000,16,8,0.267688,4,TAT-Vol2_TS_TSF0017,F
